# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
geoapify_key

'10df7cccac6c43b08ab640bcf5b7e4a9'

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,nadym,65.5333,72.5167,-9.99,94,100,4.02,RU,1706413937
1,1,lata,40.1629,-8.3327,7.73,88,67,2.13,PT,1706413938
2,2,jamestown,42.0970,-79.2353,5.01,99,100,2.57,US,1706413938
3,3,georgetown,5.4112,100.3354,32.19,65,20,3.60,MY,1706413891
4,4,waitangi,-43.9535,-176.5597,18.23,98,99,7.43,NZ,1706413938


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City"
)


# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
weather_preference = city_data_df.loc[(city_data_df['Max Temp'] <= 24) & (city_data_df['Max Temp'] >= 23) & (city_data_df['Cloudiness'] >= 40) & (city_data_df['Humidity'] >= 45)]
                 

# Drop any rows with null values
weather_preference = weather_preference.dropna()

# Display sample data
weather_preference.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,blackmans bay,-43.0167,147.3167,23.07,69,77,3.57,AU,1706413941
79,79,codrington,-38.2667,141.9667,23.75,53,77,5.58,AU,1706413959
132,132,tolanaro,-25.0319,46.9987,23.04,100,40,3.09,MG,1706413973
146,146,tondano,1.3038,124.9112,23.05,58,64,1.85,ID,1706413976
157,157,fougamou,-1.2154,10.5838,23.90,99,83,0.49,GA,1706413979


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_preference[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel = '')
hotel_df.rename(columns={'Hotel': "Hotel Name"}, inplace=True)
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,blackmans bay,AU,-43.0167,147.3167,69,
79,codrington,AU,-38.2667,141.9667,53,
132,tolanaro,MG,-25.0319,46.9987,100,
146,tondano,ID,1.3038,124.9112,58,
157,fougamou,GA,-1.2154,10.5838,99,
197,itaituba,BR,-4.2761,-55.9836,99,
199,feijo,BR,-8.1642,-70.3536,92,
202,dolisie,CG,-4.1997,12.6739,97,
310,ariquemes,BR,-9.9133,-63.0408,99,
364,alvaraes,BR,-3.2208,-64.8042,98,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
# Set the parameters for the type of place
categories = "accommodation.hotel"
limit = 20

# Set the parameters for the type of search

# set up a parameters dictionary
params = {
    "categories": categories,
    "limit": limit,
    "apiKey": geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.at[index,'Lat']
    longitude = hotel_df.at[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filters = f'circle:{longitude},{latitude},{radius}'
    bias = f"proximity:{longitude},{latitude}"
    
    params["filter"] = filters
    params["bias"] = bias
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
blackmans bay - nearest hotel: Villa Howden
codrington - nearest hotel: No hotel found
tolanaro - nearest hotel: Hôtel Mahavokey
tondano - nearest hotel: Hotel Tou Dano
fougamou - nearest hotel: Ngounié Hôtel
itaituba - nearest hotel: TG Hotel
feijo - nearest hotel: Hotel JOAFRAN
dolisie - nearest hotel: Residence Nelson
ariquemes - nearest hotel: Aquarius
alvaraes - nearest hotel: Hotel Divino
new norfolk - nearest hotel: Woodbridge on the Derwent
jiwani - nearest hotel: No hotel found
manuguru - nearest hotel: No hotel found
esmeraldas - nearest hotel: No hotel found
ormond-by-the-sea - nearest hotel: Howard Johnson
whakatane - nearest hotel: Whakatane Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
12,blackmans bay,AU,-43.0167,147.3167,69,Villa Howden
79,codrington,AU,-38.2667,141.9667,53,No hotel found
132,tolanaro,MG,-25.0319,46.9987,100,Hôtel Mahavokey
146,tondano,ID,1.3038,124.9112,58,Hotel Tou Dano
157,fougamou,GA,-1.2154,10.5838,99,Ngounié Hôtel
197,itaituba,BR,-4.2761,-55.9836,99,TG Hotel
199,feijo,BR,-8.1642,-70.3536,92,Hotel JOAFRAN
202,dolisie,CG,-4.1997,12.6739,97,Residence Nelson
310,ariquemes,BR,-9.9133,-63.0408,99,Aquarius
364,alvaraes,BR,-3.2208,-64.8042,98,Hotel Divino


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
humidity_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    hover_cols=['Country', 'Hotel Name']
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)